In [20]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns  
import re
import os

In [25]:
# フォルダのパス
folder_path = "/Users/aruhy/OneDrive/デスクトップ/data/24-3-21"

# フォルダ内のファイルのリストを取得
file_list = os.listdir(folder_path)

# ファイルごとに処理を行う
df = pd.DataFrame(columns=['V_cell','P_b','T_cell','stoichi_anode','stoichi_cathode','W_ch','D_ch','i_ave'])
for file_name in file_list:
     # ファイルの絶対パスを作成
    file_path = os.path.join(folder_path, file_name)
    
    # ファイルがディレクトリでない場合に処理を行う
    if not os.path.isdir(file_path):
        # CSVファイルのパス
        with open(file_path, 'r') as file:
            content = file.read()
            
        # 文字列の置換
        old_string = 'cd.IlMag (A/m^2) '
        new_string = '\n%'
        updated_content = content.replace(old_string, new_string)
        # ファイルを書き込みモードで開いて、更新された内容を書き込む
        with open(file_path, 'w') as file:
            file.write(updated_content)
            
        with open(file_path, 'r') as file:
            lines = file.readlines()
        # データを保持するためのリストを初期化
        data = {
            'V_cell': [],
            'P_b': [],
            'T_cell': [],
            'stoichi_anode': [],
            'stoichi_cathode': []
        }

        # データを抽出してリストに追加
        for line in lines:
            if line.startswith('%@'): #'%@'で始まる行だけTrue、つまり処理が行われる
                parts = line.split(':')
                #index = int(parts[0].split('@')[1].strip())
                #data['Index'].append(index)
                values = parts[1].strip().split(',')
                data['V_cell'].append(float(values[0].split('=')[1].strip().split(' ')[0]))
                data['P_b'].append(float(values[1].split('=')[1].strip().split(' ')[0]))
                data['T_cell'].append(float(values[2].split('=')[1].strip().split(' ')[0]))
                data['stoichi_anode'].append(float(values[3].split('=')[1].strip()))
                data['stoichi_cathode'].append(float(values[4].split('=')[1].strip().split(' ')[0]))

        # データフレームに変換
        df_param = pd.DataFrame(data)

        #W_chとD_chのカラムの追加
        W_ch = re.findall(r'W(\d\.\d+)_',file_path) # ファイル名から数字を抽出
        D_ch = re.findall(r'D(\d\.\d+)',file_path) # ファイル名から数字を抽出

        # 数字を一列のリストに格納
        data_W = W_ch * 117
        data_D = D_ch * 117

        # 117行のデータフレームを作成
        df_param2 = pd.DataFrame({'W_ch':data_W,'D_ch':data_D})

        #電流密度のデータフレームを作製
        current_row = 1
        split_pattern = re.compile(r'\s+')  # 空白文字に基づいて行を分割する正規表現パターン
        for line in lines:
            if current_row == 126:
                i_data = split_pattern.split(line.strip())
                df_i = pd.DataFrame({'i_ave':i_data})
                break #ループを抜ける
            current_row += 1

        #データフレームの結合
        df_k = pd.concat([df_param,df_param2], axis=1)
        df_k = pd.concat([df_k,df_i], axis=1)
        df = pd.concat([df,df_k], axis=0)    
# データフレームを表示
df


C:\Users\aruhy\AppData\Local\Temp\ipykernel_21844\4167953135.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df_k], axis=0)


,V_cell,P_b,T_cell,stoichi_anode,stoichi_cathode,W_ch,D_ch,i_ave
0,1.00,101330.0,373.15,1.5,2.0,0.4,0.4,10.380173171667092
1,0.95,101330.0,373.15,1.5,2.0,0.4,0.4,44.93078986452143
2,0.90,101330.0,373.15,1.5,2.0,0.4,0.4,159.31548422329277
3,0.85,101330.0,373.15,1.5,2.0,0.4,0.4,413.51944602456126
4,0.80,101330.0,373.15,1.5,2.0,0.4,0.4,819.6215377516025
...,...,...,...,...,...,...,...,...
112,0.60,101330.0,393.15,5.5,6.0,1.6,1.6,5128.186267520728
113,0.55,101330.0,393.15,5.5,6.0,1.6,1.6,6178.536866281978
114,0.50,101330.0,393.15,5.5,6.0,1.6,1.6,7269.896159764467
115,0.45,101330.0,393.15,5.5,6.0,1.6,1.6,8393.22149589775
